<a href="https://colab.research.google.com/github/Mariobahaa/Digit-Recognizer/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 ! pip install -q kaggle
 

In [0]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mariobahaa","key":"bc2430a5175201654c29c170e1c60a6f"}'}

In [0]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [0]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [0]:
 ! kaggle datasets list
 

ref                                          title                                size  lastUpdated          downloadCount  
-------------------------------------------  ----------------------------------  -----  -------------------  -------------  
chrisfilo/to-bee-or-no-to-bee                To bee or not to bee                  3GB  2020-02-04 22:55:31             41  
peijenlin/msl-m-rems-2-edr-v1.0              MSL-M-REMS-2-EDR-V1.0                11GB  2020-02-05 00:55:52              3  
prondeau/350000-jeopardy-questions           350,000+ Jeopardy Questions          19MB  2020-02-04 18:53:30             45  
prondeau/superbowlads                        Super Bowl Ads                       36KB  2020-02-04 18:04:07             91  
timoboz/can-james-bond-be-black              Can James Bond Be Black?             345B  2020-02-04 19:08:57             12  
devvret/farm-ads-binary-classification       Farm Ads Binary Classification        5MB  2020-02-04 19:12:44             11  


In [0]:
#Dataset Command
!kaggle competitions download -c digit-recognizer

  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 86.5MB/s]
  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 83.9MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 55.7MB/s]


In [0]:
! mkdir train
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [0]:
! mkdir test
! unzip test.csv.zip -d test

Archive:  test.csv.zip
  inflating: test/test.csv           


# Code Here !


In [0]:
!pip install scikit_learn

In [0]:
import pandas as pd
import tensorflow
import keras
import numpy as np
import sklearn


Using TensorFlow backend.


In [0]:
train = pd.read_csv("train/train.csv") 
test  = pd.read_csv("test/test.csv")

X = train.iloc[:,1:]
label = train.iloc[:,0]
X = np.array(X)

#print(X[:,0].size)
X = X.reshape(X[:,0].size,28,28,1)

# print(X.shape)
print(label.shape)

y = np.zeros([label.size,10])
for i in range(0,label.size):
  y[i,label[i]] = 1




(42000,)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,  Dropout, Flatten
from keras.callbacks import ModelCheckpoint


model = Sequential()
model.add(Conv2D(16,3, padding= 'same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))
model.add(Conv2D(32,3,padding= 'same', activation='relu'))
model.add(MaxPooling2D(2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath = 'MNIST1.weights.best.hdf5', verbose=1, save_best_only = True)

model.summary()
model.fit(X,y,epochs=10,verbose=1,callbacks=[checkpointer], shuffle= True, validation_split=0.3, batch_size= 64)

model.save('MNIST1.h5')







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 32)          0        

In [0]:
model.load_weights('MNIST1.weights.best.hdf5')
model.fit(X,y,epochs=1,verbose=1,callbacks=[checkpointer], shuffle= True, validation_split=0.2, batch_size= 64)

Train on 33600 samples, validate on 8400 samples
Epoch 1/1
33600/33600 [==============================] - 3s 101us/step - loss: 0.1379 - categorical_accuracy: 0.9582 - val_loss: 0.0619 - val_categorical_accuracy: 0.9840

Epoch 00001: val_loss did not improve from 0.06094


In [0]:


model.load_weights('MNIST1.weights.best.hdf5')

model.evaluate(X,y)

42000/42000 [==============================] - 2s 56us/step


[0.043725121280799306, 0.9868095238095238]

In [0]:
X_test = test.iloc[:,:]
X_test = np.array(X_test)
X_test = X_test.reshape(X_test[:,0].size,28,28,1)

pred = model.predict(X_test)
print(X_test.shape)
print(pred[0,:])

(28000, 28, 28, 1)
[1.6422617e-09 8.3406661e-07 9.9999857e-01 5.4363980e-09 1.1625433e-08
 3.5986750e-15 1.1179174e-08 4.0025787e-07 1.1349391e-07 4.4606469e-10]


In [0]:
import csv 

# names = np.zeros(pred[:,0].size)

# names = names.reshape(pred[:,0].size,1)

# for i in range (0,int(pred[:,0].size) ):
#   names[i] = i+1
#   c+=1


preds = np.zeros(pred[:,0].size)

preds = preds.reshape(pred[:,0].size,1)
for i in range (0,pred[:,0].size):
  mx = 0
  ind = 0
  for j in range(0 , 10):
    if pred[i,j] > mx:
      mx = pred[i,j]
      ind = j
  preds[i] = ind

print(preds.shape )
#print(preds)
def save(names, preds):
  c1= np.append(np.array(['ImageId']), names)
  c1= c1.reshape(len(c1),1)
  c2= np.append(np.array(['Label']), preds)
  c2= c2.reshape(len(c2),1)
  final = np.append(c1,c2, axis=1)
  with open("MNIST0.csv", "w") as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',',lineterminator='\r')
        csvWriter.writerows(final)


def save_results(names,preds):
    headRow=['ImageId','Label']
    headRow = np.expand_dims(headRow, axis=0)
    # headRow = np.transpose(headRow)
    headRow=np.array(headRow)
    preds=np.array(preds)
    names=np.array(names)
    #print('shape 1 corrct  :',np.shape(preds),'wrong shape : ',np.shape(headRow))
    final=np.concatenate((names,preds),axis=1)
    # print('final list shape :' ,np.shape(finlList))
    final=np.append(headRow,final,axis=0)
    with open("MNIST0.csv", "w") as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',',lineterminator='\r')
        csvWriter.writerows(final)

with open("MNIST2.csv", "w") as my_csv:
  writer = csv.writer(my_csv, delimiter=',',lineterminator='\r')
  writer.writerow(['ImageId','Label'])
  for i in range(0,28000):
    writer.writerow([i+1,int(preds[i])])

#save(names,preds)

(28000, 1)


In [36]:
!git config --global user.email "mariobahaa59s@gmail.com"
!git config --global user.name "mario bahaa"
 

!git init 
!git add .
!git commit -m"MNIST1 to recogize hand written digits"


Reinitialized existing Git repository in /content/.git/
On branch master
nothing to commit, working tree clean


In [35]:
#!git push --set-upstream https://github.com/Mariobahaa/Digit-Recognizer.git master
!git remote add origin https://github.com/Mariobahaa/Digit-Recognizer.git
!git push -u origin master

fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
